Esta versión no tiene nada especial, la red simplemente devuelve un vector x que supongo yo que es el minimo del problema

# Parte 1: obtenemos los datos

In [34]:
import numpy as np
import deep_inv_opt as io
import deep_inv_opt.plot as iop
import torch

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.max_open_warning'] = 0  # Let the plots flow!
%matplotlib inline

In [35]:
u_train = io.tensor(np.linspace(-1.5, 1.5, 1024).reshape((-1, 1)))
u_train

tensor([[-1.5000],
        [-1.4941],
        [-1.4883],
        [-1.4824],
        [-1.4765],
        [-1.4706],
        [-1.4648],
        [-1.4589],
        [-1.4530],
        [-1.4472],
        [-1.4413],
        [-1.4354],
        [-1.4295],
        [-1.4237],
        [-1.4178],
        [-1.4119],
        [-1.4061],
        [-1.4002],
        [-1.3943],
        [-1.3885],
        [-1.3826],
        [-1.3767],
        [-1.3708],
        [-1.3650],
        [-1.3591],
        [-1.3532],
        [-1.3474],
        [-1.3415],
        [-1.3356],
        [-1.3297],
        [-1.3239],
        [-1.3180],
        [-1.3121],
        [-1.3063],
        [-1.3004],
        [-1.2945],
        [-1.2886],
        [-1.2828],
        [-1.2769],
        [-1.2710],
        [-1.2652],
        [-1.2593],
        [-1.2534],
        [-1.2476],
        [-1.2417],
        [-1.2358],
        [-1.2299],
        [-1.2241],
        [-1.2182],
        [-1.2123],
        [-1.2065],
        [-1.2006],
        [-1.

In [44]:
u_val = io.tensor(np.linspace(-2, 2, 64).reshape((-1, 1)))
u_val

tensor([[-2.0000],
        [-1.9365],
        [-1.8730],
        [-1.8095],
        [-1.7460],
        [-1.6825],
        [-1.6190],
        [-1.5556],
        [-1.4921],
        [-1.4286],
        [-1.3651],
        [-1.3016],
        [-1.2381],
        [-1.1746],
        [-1.1111],
        [-1.0476],
        [-0.9841],
        [-0.9206],
        [-0.8571],
        [-0.7937],
        [-0.7302],
        [-0.6667],
        [-0.6032],
        [-0.5397],
        [-0.4762],
        [-0.4127],
        [-0.3492],
        [-0.2857],
        [-0.2222],
        [-0.1587],
        [-0.0952],
        [-0.0317],
        [ 0.0317],
        [ 0.0952],
        [ 0.1587],
        [ 0.2222],
        [ 0.2857],
        [ 0.3492],
        [ 0.4127],
        [ 0.4762],
        [ 0.5397],
        [ 0.6032],
        [ 0.6667],
        [ 0.7302],
        [ 0.7937],
        [ 0.8571],
        [ 0.9206],
        [ 0.9841],
        [ 1.0476],
        [ 1.1111],
        [ 1.1746],
        [ 1.2381],
        [ 1.

ahora generamos los x correspondientes del modelo real

In [45]:
class ExamplePLP(io.ParametricLP):
    # Generate an LP from a given feature vector u and weight vector w.
    def generate(self, u, w):
        c = [[torch.cos(w[0] + w[1]*u)],
             [torch.sin(w[0] + w[1]*u)]]

        A_ub = [[-1.0,  0.0],
                [ 0.0, -1.0],
                [ w[0], 1 + w[1]*u/3]]

        b_ub = [[ 0.2*w[0]*u],
                [-0.2*w[1]*u],
                [ w[0] + 0.1*u]]
        
        return c, A_ub, b_ub, None, None

In [47]:
plp_true = ExamplePLP(weights=[1.0, 1.0])

# Generate training targets by solve the true PLP at each u value.
x_train = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u_train])
torch.save(x_train, "x_train.pt")

In [48]:

x_val = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u_val])
torch.save(x_val, "x_val.pt")

In [49]:
x_val = torch.load("x_val.pt")
x_val

tensor([[ 0.4000,  1.2000],
        [ 0.3873,  1.1821],
        [ 0.3746,  1.1662],
        [ 0.3619,  1.1520],
        [ 0.3492,  1.1392],
        [ 0.3365,  1.1277],
        [ 0.3238,  1.1172],
        [ 0.3111,  1.1077],
        [ 0.2984,  1.0989],
        [ 0.2857,  1.0909],
        [ 0.2730,  1.0835],
        [ 0.2603,  1.0766],
        [ 0.2476,  1.0702],
        [ 0.2349,  1.0643],
        [ 0.2222,  1.0588],
        [ 0.2095,  1.0535],
        [ 0.1968, -0.1965],
        [ 0.1841, -0.1841],
        [ 0.1714, -0.1714],
        [ 0.1587, -0.1587],
        [ 0.1460, -0.1460],
        [ 0.1333, -0.1333],
        [ 0.1206, -0.1206],
        [ 0.1079, -0.1079],
        [ 0.0952, -0.0952],
        [ 0.0825, -0.0825],
        [ 0.0698, -0.0698],
        [ 0.0571, -0.0571],
        [ 0.0444, -0.0444],
        [ 0.0318, -0.0317],
        [ 0.0191, -0.0190],
        [ 0.0064, -0.0063],
        [-0.0063,  0.0064],
        [-0.0190,  0.0191],
        [-0.0317,  0.0318],
        [-0.0444,  0

In [51]:
x_train = torch.load("x_train.pt")
x_train

tensor([[0.3000, 1.1000],
        [0.2988, 1.0992],
        [0.2977, 1.0984],
        ...,
        [0.7035, 0.2977],
        [0.7017, 0.2988],
        [0.7000, 0.3000]], dtype=torch.float64)

# Parte 2: definimos la red y la entrenamos

In [52]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [53]:
# ver si estoy usando GPU
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [71]:
learning_rate = 1e-3
batch_size = 16
tol=1

In [72]:
# definimos el dataset
class UDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data.clone().to(dtype=torch.float32) # nota: esta dando el warning porque estoy convirtiendo un tensor a otro, en ese caso es mejor usar clone()
        # si los datos de entrada no los voy a dar como un tensor, entonces hay que poner lo que he puesto: self.data = torch.tensor(data, dtype=torch.float32), self.targets = torch.tensor(targets, dtype=torch.float32)
        self.targets = targets.clone().to(dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# Dataset con pares (u, x)
dataset = UDataset(u_train, x_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [73]:
val_dataset = UDataset(u_val, x_val)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [74]:
# funcion que resuelve el problema de programacion lineal (por ahora nos la creemos, pero hay que revisarla)
# hay que tener en cuenta que esta funcion debe realizarse con operaciones de pytorch para
# preservar el grafo de computo para poder hacer backpropagation
# def smooth_lp(c, A, b):
#     # Inicializar x con gradientes habilitados
#     x = torch.zeros(A.shape[1], requires_grad=True)
#     optimizer = torch.optim.SGD([x], lr=1e-3)

#     for _ in range(1000):
#         optimizer.zero_grad()
#         constraint_penalty = torch.sum(torch.relu(A @ x - b))
#         objective = torch.dot(c, x) + 100.0 * constraint_penalty
#         objective.backward(retain_graph=True)  # Mantén el grafo activo
#         optimizer.step()
#     return x  # Sin detach()


In [75]:
# scipy descartado porque no funciona
# voy a probar con cvxpy

import torch
import cvxpylayers.torch as cvx
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer

def solver(c_, A_, b_):
    m,n = A_.shape

    A = cp.Parameter((m, n))
    b = cp.Parameter(m)
    c = cp.Parameter(n)
    x = cp.Variable(n)

    obj = cp.Minimize(c.T @ x)
    cons = [ A @ x <= b ]
    prob = cp.Problem(obj, cons)

    layer = CvxpyLayer(prob, parameters=[c,A,b], variables=[x])

    solution, = layer(c_, A_, b_)
    return solution


In [59]:
c = torch.tensor([-1.0, -1.0])

A = torch.tensor([[1.0, 0.0],
                  [0.0, 1.0]])
b = torch.tensor([0.0,
                  0.0])

print(solver(c, A, b))

tensor([-6.1616e-06, -6.1616e-06])


Esta version es la red neuronal en la que la resolucion del problema de optimizacion esta dentro de la red

In [61]:
# definimos la red (hay que revisar la forma de la red y el por qué)
class ParametricLPNet(nn.Module):
    def __init__(self):
        super(ParametricLPNet, self).__init__()
        # Entrada de dimensión 1, salida 8 (2 para c, 4 para A, 2 para b)
        self.fc = nn.Sequential(
            nn.Linear(1, 8),
            nn.ReLU(),
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 2)
        )

    def forward(self, u):
        output = self.fc(u)
        return output


In [62]:
# funcion de perdida
# def loss_fn(rs, target):
#     '''
#     rs: lista de tensores que son las salidas de la red
#     target: lista de tensores que son las salidas deseadas

#     Esta función calcula la suma de las distancias al cuadrado entre las salidas de la red y las salidas deseadas.
#     '''
#     return torch.sum(torch.linalg.vector_norm(rs-target, ord=2, dim=1).pow(2))

loss_fn = nn.MSELoss()

In [63]:
# Crear la red neuronal
model = ParametricLPNet()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # elegir una de las dos

In [64]:
# para ver el modelo
model

ParametricLPNet(
  (fc): Sequential(
    (0): Linear(in_features=1, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=8, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=2, bias=True)
  )
)

In [65]:
# inicializamod los pesos de la red 
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='leaky_relu') # inicializamos los pesos de la red con Kaiming
        nn.init.zeros_(m.bias) # inicializamos los bias a 0

initialize_weights(model)

In [67]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    loss_media = 0.0
    accuracy = 0.0
    batch_size = dataloader.batch_size
    for batch, (u_batch, x_batch) in enumerate(dataloader):

        rs = model(u_batch)
        loss = loss_fn(rs, x_batch) # Calcular la pérdida

        # Backpropagation y optimización
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 8 == 0: # cambiar este numero para que salga cada cierto numero de iteraciones
            loss = loss.item()
            current = batch * batch_size + batch_size
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
        loss_media += loss
        accuracy += near(rs, x_batch, tol)

    return loss_media / num_batches, accuracy / size

In [68]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size=len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    #with torch.no_grad():
    for u_batch, x_batch in dataloader:
        rs = model(u_batch)
        test_loss += loss_fn(rs, x_batch).item()
        correct += near(rs, x_batch, tol) # consideramos correcto si se acerca a la solucion en la distancia euclidea 
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n\n")

In [70]:
# Entrenamiento
epochs = 1000 # poner mas
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    loss, accuracy = train_loop(dataloader, model, loss_fn, optimizer)
    print(f"\nTraining Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {loss:>8f} \n")
    
    test_loop(val_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.136345  [   32/  512]
loss: 0.177001  [  288/  512]

Training Error: 
 Accuracy: 100.0%, Avg loss: 0.138126 

Test Error: 
 Accuracy: 100.0%, Avg loss: 0.140520 


Epoch 2
-------------------------------
loss: 0.127070  [   32/  512]
loss: 0.117360  [  288/  512]

Training Error: 
 Accuracy: 100.0%, Avg loss: 0.122122 

Test Error: 
 Accuracy: 100.0%, Avg loss: 0.118671 


Epoch 3
-------------------------------
loss: 0.097086  [   32/  512]
loss: 0.112366  [  288/  512]

Training Error: 
 Accuracy: 100.0%, Avg loss: 0.112141 

Test Error: 
 Accuracy: 100.0%, Avg loss: 0.108035 


Epoch 4
-------------------------------
loss: 0.107938  [   32/  512]
loss: 0.080704  [  288/  512]

Training Error: 
 Accuracy: 100.0%, Avg loss: 0.105055 

Test Error: 
 Accuracy: 100.0%, Avg loss: 0.097675 


Epoch 5
-------------------------------
loss: 0.111450  [   32/  512]
loss: 0.116217  [  288/  512]

Training Error: 
 Accuracy: 100.0%, Avg loss: 0.097

In [ ]:
torch.save(model.state_dict(), "model_weights.pth")

In [ ]:
model(torch.tensor([[1.0]]))

tensor([[-2.3920, -2.2971]], grad_fn=<StackBackward0>)

In [ ]:
u_test = torch.tensor(np.linspace(0.1, 4, 16).reshape((-1, 1)), dtype=torch.float64)
u_test

tensor([[0.1000],
        [0.3600],
        [0.6200],
        [0.8800],
        [1.1400],
        [1.4000],
        [1.6600],
        [1.9200],
        [2.1800],
        [2.4400],
        [2.7000],
        [2.9600],
        [3.2200],
        [3.4800],
        [3.7400],
        [4.0000]], dtype=torch.float64)

ahora probamos como funciona para un x nuevo

In [ ]:
x_test = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u_test])
x_test

tensor([[5.5031e-06, 5.2914e-06],
        [5.8791e-06, 5.0125e-06],
        [6.2946e-06, 4.1114e-06],
        [1.5288e-05, 6.1703e-06],
        [6.3201e-05, 7.6848e-06],
        [2.8571e+00, 7.7249e-06],
        [2.4096e+00, 8.3817e-06],
        [2.0833e+00, 1.4402e-05],
        [1.8349e+00, 1.8074e+00],
        [1.6393e+00, 2.2303e+00],
        [6.9978e-05, 3.3749e+00],
        [1.2749e-05, 3.7000e+00],
        [6.8515e-06, 4.0250e+00],
        [1.1909e-05, 1.8498e-05],
        [2.2888e-04, 1.3794e-05],
        [9.9999e-01, 1.9565e-05]], dtype=torch.float64)

In [ ]:
model.eval()

ParametricLPNet(
  (fc): Sequential(
    (0): Linear(in_features=1, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=8, bias=True)
  )
)

In [ ]:
u = torch.tensor([[0.25]], dtype=torch.float64)
x = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u])
u,x

(tensor([[0.2500]], dtype=torch.float64),
 tensor([[5.6596e-06, 5.1630e-06]], dtype=torch.float64))

In [ ]:
model(torch.tensor([[0.25]]))

tensor([[-0.9870, -0.9380]], grad_fn=<StackBackward0>)

In [ ]:
x_pred = model(torch.tensor([[0.25]]))
x_pred

tensor([[-0.9870, -0.9380]], grad_fn=<StackBackward0>)

In [ ]:
x, x_pred, torch.norm(x - x_pred)

(tensor([[5.6596e-06, 5.1630e-06]], dtype=torch.float64),
 tensor([[-0.9870, -0.9380]], grad_fn=<StackBackward0>),
 tensor(1.3617, dtype=torch.float64, grad_fn=<LinalgVectorNormBackward0>))

Revisar:
- Función de pérdida ✔️
- Función smooth_lp (a ver si puedo sustituir la funcion smooth_lp por la funcion de linprog de deep_inv_opt)
- Organizar y explicar el código
- Estudiar por qué no disminuye el loss cuando entreno el modelo: es porque está mostrando el loss en el conjunto de test y no en el de entrenamiento ✔️
- Entrenar el modelo con más épocas y datos
- Pensar qué optimizador es mejor ✔️
- Pensar qué estructura de la red es mejor ✔️
- Inicialización de los pesos de la red ✔️
- Arreglar el porcentaje de aciertos ✔️



Bitácora: ahora lo que pasa es que da error porque el problema de optimización no está acotado, lo que tengo que hacer es revisar en el paper el intervalo donde se mueve la u porque creo que lo he puesto mal. Cuando arregle esto, tengo que volver a entrenar el modelo y ver si se está entrenando bien. También tengo que probar resolviendo el problema de optimización con la función de linprog de deep_inv_opt y con la de cvxpy a ver si alguna de las dos funciona mejor.